In [60]:
import os
from tqdm import tqdm # Para barras de progresso

from typing import List, Optional, Union, Dict

from haystack import Document
from haystack import Pipeline

from haystack_integrations.document_stores.chroma import ChromaDocumentStore
from haystack_integrations.components.embedders.ollama import OllamaDocumentEmbedder
from haystack_integrations.components.embedders.ollama import OllamaTextEmbedder
from haystack_integrations.components.retrievers.chroma import ChromaQueryTextRetriever
from haystack_integrations.components.retrievers.chroma import ChromaEmbeddingRetriever

from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.converters import PyPDFToDocument
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy

from mysql.connector import connect
from mysql.connector.connection import MySQLConnection
from mysql.connector.cursor import MySQLCursor
from mysql.connector.cursor import MySQLCursorAbstract
from mysql.connector import Error
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector.abstracts import MySQLConnectionAbstract

from sklearn.preprocessing import MinMaxScaler

import math

MYSQL_HOST = os.getenv("MYSQL_HOST", "localhost")
MYSQL_PORT = int(os.getenv("MYSQL_PORT", 3306))
MYSQL_USER = os.getenv("MYSQL_USER", "root")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD", "root")
MYSQL_DB = os.getenv("MYSQL_DB", "jogos_tabuleiro")

In [61]:
# Test SQL Connection

def get_mysql_connection() -> MySQLConnectionPool | MySQLConnectionAbstract:
  print(f"Conectando ao MySQL: host={MYSQL_HOST}, port={MYSQL_PORT}, user={MYSQL_USER}, db={MYSQL_DB}")
  return connect(
    host=MYSQL_HOST,
    user=MYSQL_USER,
    password=MYSQL_PASSWORD,
    database=MYSQL_DB,
    port=MYSQL_PORT
  )

mysql_conn: MySQLConnectionAbstract = get_mysql_connection()

Conectando ao MySQL: host=localhost, port=3306, user=root, db=jogos_tabuleiro


In [62]:
def calculate_popularity_score(
    qtd_favoritos: int,
    qtd_jogou: int,
    qtd_tem: int,
    qtd_quer: int,
    qtd_teve: int
) -> float:
    """
    Calcula um score de popularidade para um jogo usando escala logarítmica ponderada.

    A escala logarítmica é usada para normalizar os valores de entrada, que geralmente
    têm uma distribuição de cauda longa (poucos jogos muito populares, muitos com poucas interações).
    Isso evita que jogos com números massivos dominem desproporcionalmente o score.

    Args:
        qtd_favoritos: Quantidade de vezes que o jogo foi favoritado.
        qtd_jogou: Quantidade de jogadores que já jogaram.
        qtd_tem: Quantidade de jogadores que possuem o jogo.
        qtd_quer: Quantidade de jogadores que querem o jogo.
        qtd_teve: Quantidade de jogadores que já tiveram o jogo (pode ser um indicador neutro ou negativo).

    Returns:
        Um score de popularidade numérico.
    """
    # Garantir que todos os inputs sejam numéricos e não nulos, tratando None como 0.
    qtd_favoritos = qtd_favoritos or 0
    qtd_jogou = qtd_jogou or 0
    qtd_tem = qtd_tem or 0
    qtd_quer = qtd_quer or 0
    qtd_teve = qtd_teve or 0

    # Pesos definidos para cada métrica. A soma não precisa ser 1.
    # Favoritos e Jogou têm o maior peso, pois são indicadores fortes de engajamento.
    # Teve tem um peso negativo, pois pode indicar que os jogadores se desfizeram do jogo.
    weights = {
        "favoritos": 0.4,
        "jogou": 0.3,
        "tem": 0.15,
        "quer": 0.1,
        "teve": -0.05  # Peso negativo para a métrica "teve"
    }

    # Aplicar transformação logarítmica (log1p para lidar com valores 0)
    # log1p(x) é equivalente a log(1 + x)
    log_favoritos = math.log1p(qtd_favoritos)
    log_jogou = math.log1p(qtd_jogou)
    log_tem = math.log1p(qtd_tem)
    log_quer = math.log1p(qtd_quer)
    log_teve = math.log1p(qtd_teve)

    # Calcular o score ponderado final
    popularity_score = (
        (log_favoritos * weights["favoritos"]) +
        (log_jogou * weights["jogou"]) +
        (log_tem * weights["tem"]) +
        (log_quer * weights["quer"]) +
        (log_teve * weights["teve"])
    )
    
    # Garantir que o score não seja negativo (pode acontecer se 'qtd_teve' for muito alto)
    return max(0, popularity_score)

In [ ]:
def get_mysql_connection() -> MySQLConnectionPool | MySQLConnectionAbstract:
  print(f"Conectando ao MySQL: host={MYSQL_HOST}, port={MYSQL_PORT}, user={MYSQL_USER}, db={MYSQL_DB}")
  return connect(
    host=MYSQL_HOST,
    user=MYSQL_USER,
    password=MYSQL_PASSWORD,
    database=MYSQL_DB,
    port=MYSQL_PORT
  )

def fetch_boardgames_from_mysql(conn: MySQLConnectionAbstract) -> list[dict]:
  print("Buscando jogos do MySQL...")
  cursor: MySQLCursorAbstract = conn.cursor(dictionary=True)

  query: str = """
    SELECT
        jd.id, 
        jd.nm_jogo, 
        jd.idade_minima, 
        jd.qt_favorito, 
        jd.qt_jogadores_max,
        jd.qt_jogadores_min,
        jd.qt_jogou,
        jd.qt_quer,
        jd.qt_tem,
        jd.qt_teve,
        jd.thumb,
        jd.tp_jogo,
        jd.vl_tempo_jogo,
        GROUP_CONCAT(DISTINCT c.nm_categoria SEPARATOR ', ') as categorias,
        GROUP_CONCAT(DISTINCT t.nm_tema SEPARATOR ', ') as temas,
        GROUP_CONCAT(DISTINCT a.nm_profissional SEPARATOR ', ') as artistas,
        GROUP_CONCAT(DISTINCT m.nm_mecanica SEPARATOR ', ') as mecanicas
    FROM jogo_detalhado jd
    LEFT JOIN jogo_categoria jc ON jc.jogo_id = jd.id
    LEFT JOIN categoria c ON c.id = jc.categoria_id
    LEFT JOIN jogo_tema jt ON jt.jogo_id = jd.id
    LEFT JOIN tema t ON jt.tema_id = t.id
    LEFT JOIN jogo_artista ja ON ja.jogo_id = jd.id
    LEFT JOIN artista a ON a.id = ja.artista_id
    LEFT JOIN jogo_mecanica jm ON jm.jogo_id = jd.id
    LEFT JOIN mecanica m ON jm.mecanica_id = m.id
    WHERE jd.id IS NOT NULL AND jd.nm_jogo IS NOT NULL AND jd.nm_jogo != '' AND jd.tp_jogo = 'b'
    GROUP BY jd.id, jd.nm_jogo, jd.idade_minima, jd.qt_favorito, jd.qt_jogadores_max,
              jd.qt_jogadores_min, jd.qt_jogou, jd.qt_quer, jd.qt_tem, jd.qt_teve,
              jd.thumb, jd.tp_jogo, jd.vl_tempo_jogo
  """
  
  cursor.execute(query)
  results: List[RowType | Dict[str, RowItemType]] = cursor.fetchall()
  cursor.close()
  print(f"Total de {len(results)} jogos buscados do MySQL.")
  return results

mysql_conn: MySQLConnectionAbstract = get_mysql_connection()
boardgames: list[dict] = fetch_boardgames_from_mysql(mysql_conn)
mysql_conn.close()

print(f"Total de jogos encontrados: {len(boardgames)}")
print(f"Exemplo de jogo: {boardgames[0]}")

Conectando ao MySQL: host=localhost, port=3306, user=root, db=jogos_tabuleiro
Buscando jogos do MySQL...
Total de 1000 jogos buscados do MySQL.
Total de jogos encontrados: 1000
Exemplo de jogo: {'id': 1, 'nm_jogo': 'Old Dragon 2ª Edição: Livro I - Regras Básicas', 'idade_minima': 12, 'qt_favorito': 69, 'qt_jogadores_max': 10, 'qt_jogadores_min': 1, 'qt_jogou': 74, 'qt_quer': 57, 'qt_tem': 204, 'qt_teve': 1, 'thumb': 'https://storage.googleapis.com/ludopedia-capas/43761_t.jpg', 'tp_jogo': 'b', 'vl_tempo_jogo': 120, 'categorias': None, 'temas': 'Aventura, Fantasia, Literatura, Medieval', 'artistas': None, 'mecanicas': 'Narração de Histórias, Papel e Caneta, Rolagem de Dados, RPG'}


In [64]:
def prepare_haystack_documents(boardgames_data: list[dict]) -> list[Document]:
    """
    Prepara uma lista de Documentos Haystack a partir dos dados dos jogos de tabuleiro.
    Cada Documento contém informações relevantes sobre o jogo, como nome, tipo, idade mínima,
    número de jogadores, tempo de jogo, categorias, temas, mecânicas e artistas.
    
    Args:
        boardgames_data (list[dict]): Lista de dicionários contendo os dados dos jogos de tabuleiro.
    
    Returns:
        list[Document]: Lista de Documentos Haystack preparados.
    """
    
    print("Preparando Documentos Haystack...")
    haystack_docs: list[Document] = []

    for game in tqdm(boardgames_data, desc="Convertendo dados para Documentos"):
        content_parts: list[str] = [
            f"Nome do jogo: {game.get('nm_jogo', 'N/A')}.",
            f"Tipo: {game.get('tp_jogo', 'N/A')}.",
            f"Adequado para maiores de {game.get('idade_minima', 'N/A')} anos.",
            f"Pode ser jogado por {game.get('qt_jogadores_min', 'N/A')} a {game.get('qt_jogadores_max', 'N/A')} jogadores.",
            f"Tempo médio de jogo: {game.get('vl_tempo_jogo', 'N/A')} minutos.",
            f"Descrição: {game.get('descricao', 'N/A')}." if game.get('descricao') else "Descrição não disponível.",
            f"O jogo se baseia nas seguintes categorias: {game.get('categorias', 'N/A')}." if game.get('categorias') else "Categorias não disponíveis.",
            f"O jogo aborda os seguintes temas: {game.get('temas', 'N/A')}." if game.get('temas') else "Temas não disponíveis.",
            f"O jogo utiliza as seguintes mecânicas: {game.get('mecanicas', 'N/A')}." if game.get('mecanicas') else "Mecânicas não disponíveis.",
        ]
        
        if game.get('categorias'):
            content_parts.append(f"Categorias: {game['categorias']}.")
        if game.get('temas'):
            content_parts.append(f"Temas: {game['temas']}.")
        if game.get('mecanicas'):
            content_parts.append(f"Mecânicas: {game['mecanicas']}.")
        if game.get('artistas'):
            content_parts.append(f"Artistas: {game['artistas']}.")

        content: str = " ".join(content_parts)
        
        score: float = calculate_popularity_score(
            game.get('qt_favorito'),
            game.get('qt_jogou'),
            game.get('qt_tem'),
            game.get('qt_quer'),
            game.get('qt_teve')
        )

        meta: Dict[str, Optional[Union[str, int]]] = {
            "mysql_id": game.get('id'),
            "title": game.get('nm_jogo', 'N/A'), # 'title' é um campo comum para meta
            "min_age": game.get('idade_minima'),
            "max_players": game.get('qt_jogadores_max'),
            "min_players": game.get('qt_jogadores_min'),
            "play_time_minutes": game.get('vl_tempo_jogo'),
            "thumbnail": game.get('thumb'),
            "game_type": game.get('tp_jogo'),
            "categories_list": game.get('categorias', '').split(', ') if game.get('categorias') else [],
            "themes_list": game.get('temas', '').split(', ') if game.get('temas') else [],
            "mechanics_list": game.get('mecanicas', '').split(', ') if game.get('mecanicas') else [],
            "artists_list": game.get('artistas', '').split(', ') if game.get('artistas') else [],
            "favorite_count": game.get('qt_favorito'),
            "played_count": game.get('qt_jogou'),
            "want_count": game.get('qt_quer'),
            "have_count": game.get('qt_tem'),
            "had_count": game.get('qt_teve'),
            "popularity_score": score
        }
        
        # Remove chaves com valor None dos metadados para evitar problemas com alguns DocumentStores
        meta_cleaned: Dict[str, str | int] = {k: v for k, v in meta.items() if v is not None}

        haystack_docs.append(Document(content=content, meta=meta_cleaned))

    print(f"Total de {len(haystack_docs)} Documentos Haystack preparados.")
    if haystack_docs:
        print("\nAmostra do primeiro Documento Haystack:")
        print(f"Content: {haystack_docs[0].content[:500]}...")
        print(f"Meta: {haystack_docs[0].meta}")
    return haystack_docs

In [65]:
print("Inicializando o ChromaDocumentStore...")

document_store_for_raw_boardgames: ChromaDocumentStore = ChromaDocumentStore(persist_path="./test_databases/chroma_db_raw_boardgames", collection_name="raw_boardgames")

print(f"ChromaDocumentStore inicializado com persist_path: {document_store_for_raw_boardgames._persist_path} e collection_name: {document_store_for_raw_boardgames._collection_name}")

print(f"ChromaDocumentStore usando path: ./test_databases/chroma_db_raw_boardgames. Documentos existentes: {document_store_for_raw_boardgames.count_documents()}")

Inicializando o ChromaDocumentStore...
ChromaDocumentStore inicializado com persist_path: ./test_databases/chroma_db_raw_boardgames e collection_name: raw_boardgames
ChromaDocumentStore usando path: ./test_databases/chroma_db_raw_boardgames. Documentos existentes: 0


In [66]:
# Embedders
document_embedder: OllamaDocumentEmbedder = OllamaDocumentEmbedder(
  model="nomic-embed-text",
  url="http://localhost:11434",
  # Você pode adicionar outros parâmetros como timeout, etc.
      # generation_kwargs={"num_gpu": 1} # Exemplo se quiser especificar uso de GPU no Ollama
)

text_embedder: OllamaTextEmbedder = OllamaTextEmbedder(
  model="nomic-embed-text",
  url="http://localhost:11434",
  # Você pode adicionar outros parâmetros como timeout, etc.
      # generation_kwargs={"num_gpu": 1} # Exemplo se quiser especificar uso de GPU no Ollama
)

In [67]:
# Retrievers
query_text_retriever: ChromaQueryTextRetriever = ChromaQueryTextRetriever(
    document_store=document_store_for_raw_boardgames,
)

embedding_retriever: ChromaEmbeddingRetriever = ChromaEmbeddingRetriever(
    document_store=document_store_for_raw_boardgames,
)

In [68]:
connection_mysql: MySQLConnectionAbstract = get_mysql_connection()
raw_boardgames: list[dict] = fetch_boardgames_from_mysql(connection_mysql)
haystack_docs: list[Document] = prepare_haystack_documents(raw_boardgames)

print(f"Total de Documentos a serem escritos no ChromaDocumentStore: {len(haystack_docs)}")

Conectando ao MySQL: host=localhost, port=3306, user=root, db=jogos_tabuleiro
Buscando jogos do MySQL...
Total de 1000 jogos buscados do MySQL.
Preparando Documentos Haystack...


Convertendo dados para Documentos: 100%|██████████| 1000/1000 [00:00<00:00, 38658.60it/s]

Total de 1000 Documentos Haystack preparados.

Amostra do primeiro Documento Haystack:
Content: Nome do jogo: Old Dragon 2ª Edição: Livro I - Regras Básicas. Tipo: b. Adequado para maiores de 12 anos. Pode ser jogado por 1 a 10 jogadores. Tempo médio de jogo: 120 minutos. Descrição não disponível. Categorias não disponíveis. O jogo aborda os seguintes temas: Aventura, Fantasia, Literatura, Medieval. O jogo utiliza as seguintes mecânicas: Narração de Histórias, Papel e Caneta, Rolagem de Dados, RPG. Temas: Aventura, Fantasia, Literatura, Medieval. Mecânicas: Narração de Histórias, Papel e C...
Meta: {'mysql_id': 1, 'title': 'Old Dragon 2ª Edição: Livro I - Regras Básicas', 'min_age': 12, 'max_players': 10, 'min_players': 1, 'play_time_minutes': 120, 'thumbnail': 'https://storage.googleapis.com/ludopedia-capas/43761_t.jpg', 'game_type': 'b', 'categories_list': [], 'themes_list': ['Aventura', 'Fantasia', 'Literatura', 'Medieval'], 'mechanics_list': ['Narração de Histórias', 'Papel e Canet

In [69]:
try:
  connection_mysql: MySQLConnectionAbstract = get_mysql_connection()
  raw_boardgames: list[dict] = fetch_boardgames_from_mysql(connection_mysql)
  haystack_docs: list[Document] = prepare_haystack_documents(raw_boardgames)
finally:
  if connection_mysql and connection_mysql.is_connected():
    connection_mysql.close()
    print("Conexão MySQL fechada.")
  
if not haystack_docs:
    print("Nenhum documento Haystack foi preparado. Encerrando o script.")
else:
  print("Inicializando componentes Haystack para indexação...")
  
  print(f"DocumentEmbedder inicializado com modelo: {document_embedder.model} e URL: {document_embedder.url}")
  document_embedder.run(haystack_docs)
  print("Documentos preparados para indexação.")
  
  # Opcional: Limpeza e Divisão de Documentos
  # cleaner = DocumentCleaner() # Remove cabeçalhos/rodapés, linhas em branco etc. (pode não ser necessário)
  # splitter = DocumentSplitter(split_by="word", split_length=200, split_overlap=20) # Divide documentos longos

  writer: DocumentWriter = DocumentWriter(
    document_store=document_store_for_raw_boardgames,
    policy=DuplicatePolicy.OVERWRITE,
  )
  
  # Pipeline de indexação
  indexing_pipeline: Pipeline = Pipeline()
  # Adicione cleaner e splitter se decidir usá-los:
  # indexing_pipeline.add_component("cleaner", cleaner)
  # indexing_pipeline.add_component("splitter", splitter)
  indexing_pipeline.add_component("embedder", document_embedder)
  indexing_pipeline.add_component("writer", writer)
  
  indexing_pipeline.connect("embedder.documents", "writer.documents")
  
  print("Executando pipeline de indexação...")
  
  batch_size: int = 256
  
  print(f"Processando {len(haystack_docs)} documentos em lotes de {batch_size}...")
  for i in tqdm(range(0, len(haystack_docs), batch_size), desc="Indexando documentos"):
    batch_docs: List[Document] = haystack_docs[i:i + batch_size]
    indexing_pipeline.run({
      "embedder": {"documents": batch_docs},
    })
    print(f"Processando lote {i // batch_size + 1} de {len(haystack_docs) // batch_size + 1}...") 
    print(f"Lote {i // batch_size + 1} processado. Total de documentos indexados: {document_store_for_raw_boardgames.count_documents()}")
  print("Pipeline de indexação concluída.")
  print(f"Total de documentos no ChromaDB ({document_store_for_raw_boardgames._collection_name}): {document_store_for_raw_boardgames.count_documents()}")

Conectando ao MySQL: host=localhost, port=3306, user=root, db=jogos_tabuleiro
Buscando jogos do MySQL...
Total de 1000 jogos buscados do MySQL.
Preparando Documentos Haystack...


Convertendo dados para Documentos: 100%|██████████| 1000/1000 [00:00<00:00, 24942.64it/s]


Total de 1000 Documentos Haystack preparados.

Amostra do primeiro Documento Haystack:
Content: Nome do jogo: Old Dragon 2ª Edição: Livro I - Regras Básicas. Tipo: b. Adequado para maiores de 12 anos. Pode ser jogado por 1 a 10 jogadores. Tempo médio de jogo: 120 minutos. Descrição não disponível. Categorias não disponíveis. O jogo aborda os seguintes temas: Aventura, Fantasia, Literatura, Medieval. O jogo utiliza as seguintes mecânicas: Narração de Histórias, Papel e Caneta, Rolagem de Dados, RPG. Temas: Aventura, Fantasia, Literatura, Medieval. Mecânicas: Narração de Histórias, Papel e C...
Meta: {'mysql_id': 1, 'title': 'Old Dragon 2ª Edição: Livro I - Regras Básicas', 'min_age': 12, 'max_players': 10, 'min_players': 1, 'play_time_minutes': 120, 'thumbnail': 'https://storage.googleapis.com/ludopedia-capas/43761_t.jpg', 'game_type': 'b', 'categories_list': [], 'themes_list': ['Aventura', 'Fantasia', 'Literatura', 'Medieval'], 'mechanics_list': ['Narração de Histórias', 'Papel e Canet

Calculating embeddings: 100%|██████████| 32/32 [09:09<00:00, 17.16s/it]


Documentos preparados para indexação.
Executando pipeline de indexação...
Processando 1000 documentos em lotes de 256...


...

Processando lote 2 de 4...
Lote 2 processado. Total de documentos indexados: 512


Calculating embeddings: 100%|██████████| 8/8 [02:09<00:00, 16.18s/it]
Document 79817a83996d27ce6152d7c09d90f364050a174cd6a29a10251e1773f314df0d contains `meta` values of unsupported types for the keys: categories_list, themes_list, mechanics_list, artists_list. These items will be discarded. Supported types are: str, int, float, bool.
Document 8acd7e882acf584461c612c4448c56529586b725b95ef235dc9a1844049fd603 contains `meta` values of unsupported types for the keys: categories_list, themes_list, mechanics_list, artists_list. These items will be discarded. Supported types are: str, int, float, bool.
Document 3354ae97732f1e3531c8b460727f395b89a81b9e2dbedbcb15b2f2c1f71b66a9 contains `meta` values of unsupported types for the keys: categories_list, themes_list, mechanics_list, artists_list. These items will be discarded. Supported types are: str, int, float, bool.
Document e4404e4a74a458c3fad76ade5418485b3aa3c73b6fcc6255e795659d90a53eb4 contains `meta` values of unsupported types for the key

Processando lote 3 de 4...
Lote 3 processado. Total de documentos indexados: 768


Calculating embeddings: 100%|██████████| 8/8 [02:02<00:00, 15.26s/it]
Document 0f36b0be442c0f36a502dbeb4ee61f7441a3e15cc30c73a168bfddc5c76b8c04 contains `meta` values of unsupported types for the keys: categories_list, themes_list, mechanics_list, artists_list. These items will be discarded. Supported types are: str, int, float, bool.
Document fdf574d8a59ee8e7b93026d1333e9dea986f0e2df3432ebaca9a965b9e07420d contains `meta` values of unsupported types for the keys: categories_list, themes_list, mechanics_list, artists_list. These items will be discarded. Supported types are: str, int, float, bool.
Document f708e443cc53d48bf19afa118d774f482c03dbb1505621563b45acb41210c428 contains `meta` values of unsupported types for the keys: categories_list, themes_list, mechanics_list, artists_list. These items will be discarded. Supported types are: str, int, float, bool.
Document 3b4b2c282c41b7f1b6aac3d08f1873402c6fa7660a7ecd614ce25da949009fd6 contains `meta` values of unsupported types for the key

Processando lote 4 de 4...
Lote 4 processado. Total de documentos indexados: 1000
Pipeline de indexação concluída.
Total de documentos no ChromaDB (raw_boardgames): 1000


In [70]:
print(f"ChromaDocumentStore para teste de query inicializado com persist_path: {document_store_for_raw_boardgames._persist_path} e collection_name: {document_store_for_raw_boardgames._collection_name}")

query_text_retriever: ChromaQueryTextRetriever = ChromaQueryTextRetriever(
    document_store=document_store_for_raw_boardgames,
)

print("\nTestando Opção 2: OllamaTextEmbedder + ChromaEmbeddingRetriever")
text_embedder = OllamaTextEmbedder(
    model="nomic-embed-text", 
    url="http://localhost:11434"
)

embedding_retriever = ChromaEmbeddingRetriever(document_store=document_store_for_raw_boardgames)

querying_pipeline_option2: Pipeline = Pipeline()
querying_pipeline_option2.add_component("text_embedder", text_embedder)
querying_pipeline_option2.add_component("embedding_retriever", embedding_retriever)
querying_pipeline_option2.connect("text_embedder.embedding", "embedding_retriever.query_embedding")

results_option2 = querying_pipeline_option2.run({
    "text_embedder": {"text": "jogo de estratégia para dois jogadores com tema medieval"},
    "embedding_retriever": {"top_k": 3}
})

if results_option2["embedding_retriever"]["documents"]:
    print("Resultados da Opção 2:")
    for doc in results_option2["embedding_retriever"]["documents"]:
        print(f"  Title: {doc.meta.get('title', 'N/A')}, Score: {doc.score:.4f}")
        print(f"  Content snippet: {doc.content[:200]}...")
        print(f"  Meta: {doc.meta}")
        print("\n")
else:
    print("Nenhum resultado encontrado pela Opção 2.")

ChromaDocumentStore para teste de query inicializado com persist_path: ./test_databases/chroma_db_raw_boardgames e collection_name: raw_boardgames

Testando Opção 2: OllamaTextEmbedder + ChromaEmbeddingRetriever
Resultados da Opção 2:
  Title: Torres, Score: 398.6153
  Content snippet: Nome do jogo: Torres. Tipo: b. Adequado para maiores de 12 anos. Pode ser jogado por 2 a 4 jogadores. Tempo médio de jogo: 60 minutos. Descrição não disponível. O jogo se baseia nas seguintes categori...
  Meta: {'favorite_count': 24, 'had_count': 121, 'play_time_minutes': 60, 'popularity_score': 4.370976742970466, 'played_count': 514, 'game_type': 'b', 'max_players': 4, 'want_count': 331, 'title': 'Torres', 'thumbnail': 'https://storage.googleapis.com/ludopedia-capas/939_t.jpg', 'min_age': 12, 'mysql_id': 854, 'have_count': 329, 'min_players': 2}


  Title: Xadrez, Score: 398.7942
  Content snippet: Nome do jogo: Xadrez. Tipo: b. Adequado para maiores de 6 anos. Pode ser jogado por 2 a 2 jogadores. Temp